# Практическое задание

## Данные о студенте

1. **ФИО**: Айрапетьянц Каринэ Арсеновна
2. **Факультет**: ВМК
3. **Курс**: 5
4. **Группа**: 519/2

## Замечания

* Заполненный ноутбук необходимо сдать боту
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

## Реализация собственного нейросетевого пакета для запуска и обучения нейронных сетей

Задание состоит из трёх частей:
1. Реализация прямого вывода нейронной сети (5 баллов)
2. Реализация градиентов по входу и распространения градиента по сети (5 баллов)
3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети (10 баллов)

Дополнительные баллы можно получить при реализации обучения сети со свёрточными слоями (10 баллов), с транспонированной свёрткой (10 баллов), дополнительного оптимизатора (5 баллов). 

###  1. Реализация вывода собственной нейронной сети

1.1 Внимательно ознакомьтесь с интерфейсом слоя. Любой слой должен содержать как минимум три метода:
- конструктор
- прямой вывод 
- обратный вывод, производные по входу и по параметрам

In [1]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'       
    def forward(self, input_data):
        pass
    def backward(self, input_data):
        return [self.grad_x(input_data), self.grad_param(input_data)]
    
    def grad_x(self, input_data):
        pass
        
    def grad_param(self, input_data):
        return []
    
    def update_param(self, grads, learning_rate):
        pass


1.2 Ниже предствален интерфейс класса  Network. Обратите внимание на реализацию метода predict, который последовательно обрабатывает входные данные слой за слоем.

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

class Network(object):
    def __init__(self, layers, loss=None):
        self.name = 'Network'
        self.layers = layers
        self.loss = loss

    def forward(self, input_data):
        return self.predict(input_data)

    def grad_x(self, input_data, labels):
        
        # curr_data = self.loss.grad_x(self.forward(input_data), labels)
        # derivs = [curr_data]
        #
        # for i in range(len(self.layers) - 1, -1, -1):
        #     curr_data = self.layers[i].grad_x(curr_data)
        #     derivs.append(curr_data)
        #
        # 

        n = len(self.layers)
        derivs = [[] for i in range(n + 1)]
        j = n

        curr_data = input_data
        for layer in self.layers:
            derivs[j] = layer.grad_x(curr_data)
            curr_data = layer.forward(curr_data)
            j -= 1

        derivs[j] = self.loss.grad_x(curr_data, labels)
        
        
        der = self.__calc_der(derivs, -1, input_data.shape[0])
        return der
    

    def grad_param(self, input_data, labels):
        
        n = len(self.layers)
        p_grads, p, d = [], [], [[] for i in range(n + 1)]
        j = n

        curr_data = input_data
        for i, layer in enumerate(self.layers):
            d[j] = layer.grad_x(curr_data)
            next_data = layer.forward(curr_data)
            p.append(layer.grad_param(curr_data))
            curr_data = next_data
            j -= 1

        d[j] = self.loss.grad_x(curr_data, labels)

        for i, p_grad in enumerate(p):
            if len(p_grad) != 0:
                der = self.__calc_der(d, i, input_data.shape[0])

                der = np.expand_dims(der, 1)

                for b in range(input_data.shape[0]):
                    for g in range(len(p_grad)):
                        p_grad[g][b] = np.dot(der[b], p_grad[g][b])
            p_grads.append(p_grad)

        return p_grads


    def update(self, grad_list, learning_rate):
        for i in range(len(self.layers)):
            self.layers[i].update_param(grad_list[i], learning_rate)

    def predict(self, input_data):
        current_input = input_data
        for layer in self.layers:
            current_input = layer.forward(current_input)
        return current_input

    def calculate_loss(self, input_data, labels):
        return self.loss.forward(self.forward(input_data), labels)

    def train_step(self, input_data, labels, learning_rate=0.001):
        grad_list = self.grad_param(input_data, labels)
        self.update(grad_list, learning_rate)

    def fit(self, trainX, trainY, validation_split=0.25,
            batch_size=1, nb_epoch=1, learning_rate=0.01):

        train_x, val_x, train_y, val_y = train_test_split(trainX, trainY,
                                                          test_size=validation_split,
                                                          random_state=42)
        for epoch in range(nb_epoch):
            # train one epoch
            for i in tqdm(range(int(len(train_x) / batch_size))):
                batch_x = train_x[i * batch_size: (i + 1) * batch_size]
                batch_y = train_y[i * batch_size: (i + 1) * batch_size]
                self.train_step(batch_x, batch_y, learning_rate)
            # validate
            val_accuracy = self.evaluate(val_x, val_y)
            print('%d epoch: val %.2f' % (epoch + 1, val_accuracy))

    def evaluate(self, testX, testY):
        y_pred = np.argmax(self.predict(testX), axis=1)
        y_true = np.argmax(testY, axis=1)
        val_accuracy = np.sum((y_pred == y_true)) / (len(y_true))
        return val_accuracy

    
    def __calc_der(self, derivs, k, n):
        der = []
        for b in range(n):
            der.append(derivs[0][b])
            for j in range(1, len(derivs) - k - 1, 1):
                der[b] = np.dot(der[b], derivs[j][b])

        return np.array(der)



#### 1.1 Необходимо реализовать метод forward для вычисления следующих слоёв:

- DenseLayer
- ReLU
- Softmax
- FlattenLayer

In [3]:
#импорты
import numpy as np

In [4]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
        super().__init__()
        self.name = 'Dense'
        self.input_dim = input_dim
        self.output_dim = output_dim
        if W_init is None or b_init is None:
            lecun_init = np.sqrt(3) / np.sqrt(input_dim)
            self.W = np.random.uniform(-lecun_init, lecun_init, (input_dim, output_dim))
            self.b = np.zeros(output_dim, 'float32')
        else:
            self.W = W_init
            self.b = b_init

    def forward(self, input_data):
        return input_data@self.W + self.b

    def grad_x(self, input_data):
        res = []
        for b in range(input_data.shape[0]):
            res.append(self.W.T)
        return np.array(res)

    def grad_b(self, input_data):
        res = []
        for b in range(input_data.shape[0]):
            res.append(np.eye(self.output_dim))
        return np.array(res)

    def grad_W(self, input_data):
        # медленно!
        # res=[]
        # for x in input_data:
        #     inter_res = []
        #     for el in x:
        #         to_concat = np.diag(np.ones(len(x))*el)
        #         if len(inter_res) == 0:
        #             inter_res = to_concat
        #             continue
        #         inter_res = np.concatenate((inter_res, to_concat), axis=1)
        #
        #     res.append(inter_res)

        # return np.array(res)

        res = np.zeros((input_data.shape[0], self.output_dim, self.input_dim * self.output_dim))

        for b in range(input_data.shape[0]):
            for i in range(self.output_dim):
                for j in range(self.input_dim):
                    res[b][i][i + j * self.output_dim] = input_data[b][j]

        return res

    def update_W(self, grad, learning_rate):
        for b in range(grad.shape[0]):
            self.W -= learning_rate * np.mean(grad[b], axis=0).reshape(self.W.shape)

    def update_b(self, grad, learning_rate):
        for bt in range(grad.shape[0]):
            self.b -= learning_rate * np.mean(grad[bt], axis=0)

    def update_param(self, params_grad, learning_rate):
        self.update_W(params_grad[0], learning_rate)
        self.update_b(params_grad[1], learning_rate)

    def grad_param(self, input_data):
        return [self.grad_W(input_data), self.grad_b(input_data)]


class ReLU(Layer):
    def __init__(self):
        super().__init__()
        self.name = 'ReLU'

    def forward(self, input_data):
        return np.maximum(input_data, 0)

    def grad_x(self, input_data):
        derivs = np.where(input_data > 0, 1.0, 0.0)
        return np.asarray([np.diag(der) for der in derivs])


class Softmax(Layer):
    def __init__(self):
        super().__init__()
        self.name = 'Softmax'

    def forward(self, input_data): 
        return np.array([np.exp(input_data[b]) / np.sum(np.exp(input_data[b])) for b in range(input_data.shape[0])])

    def grad_x(self, input_data):        
        # s'= s(x_i)-s(x_i)s(x_j), i=j || 0-s(x_i)s(x_j), i!=j => diag(s)-[s(x_i)s(x_j)], i=1,..n, j=1,..,n
        input_data = self.forward(input_data)
        res = []
        for b in range(input_data.shape[0]):
            res.append(np.diag(input_data[b]) - np.outer(input_data[b], input_data[b]))        
        return np.array(res)


class FlattenLayer(Layer):
    def __init__(self):
        super().__init__()
        self.name = 'Flatten'
        self._prev_shape = None

    def forward(self, input_data):
        self._prev_shape = input_data.shape
        return np.reshape(input_data, (input_data.shape[0], -1))

    def grad_x(self, input_data):
        return np.reshape(input_data, self._prev_shape)



#### 1.2 Реализуйте теперь свёрточный слой и транспонированную свёртку  (опционально)

#### Для сверточного слоя успела реализовать только прямой проход и его тестирование. 

Напоминание: 'same' padding: $2p=(W-1)s-W+ksize$, аналогично для $H$

In [5]:
class Conv2DLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding='same', stride=1, K_init=None, b_init=None):
        # padding: 'same' или 'valid'
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # Работаем с единообразным сдвигом, поэтому stride - одно число
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2D'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride
        
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, height, width, input_channels]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out 
            

        assert input_data.shape[3]==self.kernel.shape[2], 'Input in_channels!=kernel in_channels!'
        
    
        def calc_conv(tensor, out_size, steps):
            
            Y = np.zeros((tensor.shape[0], out_size[0], out_size[1], self.output_channels))

            for batch in range(tensor.shape[0]):

                for cout in range(self.output_channels):  
                    
                    cards = np.zeros((out_size[0], out_size[1], self.input_channels))

                    for cin in range(self.input_channels):
                        
                        card = np.zeros((out_size[0], out_size[1]))
                        
                        for i in range(steps[0]): 
                            for j in range(steps[1]):  
                                to_mul = tensor[batch,i*self.stride:self.kernel_size+i*self.stride,
                                                j*self.stride:self.kernel_size+j*self.stride,cin]
                                card[i][j] = np.sum(to_mul*self.kernel[:,:,cin,cout])


                        cards[:,:,cin] = card

                    Y[batch,:,:,cout]=np.sum(cards, axis=2)+self.bias[cout] 

            return Y 
       
    
        
        H, W = input_data.shape[1], input_data.shape[2]
       
        if self.padding=='valid':

            H_out = int((H-self.kernel_size)/self.stride+1)
            W_out = int((W-self.kernel_size)/self.stride+1)

            return calc_conv(input_data,(H_out, W_out), (H_out, W_out))


        if self.padding=='same':
          
            p_W=((W-1)*self.stride-W+self.kernel_size) 
            p_H=((H-1)*self.stride-H+self.kernel_size)

            pad_top = int(p_H//2)
            pad_bottom = int(p_H - pad_top) 
            pad_left = int(p_W//2)              
            pad_right = int(p_W - pad_left)

        
            padded_tensor = np.zeros((input_data.shape[0], input_data.shape[1] + int(p_H), 
                                      input_data.shape[2] + int(p_W), input_data.shape[3]))

            padded_tensor[:,pad_top:-pad_bottom, pad_left:-pad_right, :] = input_data


            H_steps = int((padded_tensor.shape[1]-self.kernel_size)/self.stride+1)    
            W_steps = int((padded_tensor.shape[2]-self.kernel_size)/self.stride+1)

            return calc_conv(padded_tensor, (H,W), (H_steps, W_steps))



            

    def grad_x(self, input_data):
        pass
    def grad_kernel(self, input_data):
        pass
        
    def grad_param(self, input_data):
        return [self.grad_kernel(input_data)]
    
    def update_param(self, grads, learning_rate):   
        # smth like this 
        self.kernel-=learning_rate*np.mean(grads[0], axis=0)






In [6]:
class Conv2DTrLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding=0, stride=1, K_init=None, b_init=None):      
        # padding: число (сколько отрезать от модифицированной входной карты)
        # Работаем с квадратными ядрами, поэтому kernel_size - одно число
        # stride - одно число (коэффициент расширения)
        # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
        self.name = 'Conv2DTr'
        self.kernel_size = kernel_size
        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel = K_init
        self.bias = b_init
        self.padding = padding
        self.stride = stride
    def forward(self, input_data):
        # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
        # Вначале нужно проверить на согласование размерностей входных данных и ядра!
        # Нужно заполнить Numpy-тензор out 
        out = np.empty([])
        return out

    
    def grad_x(self):
        pass
    def grad_kernel(self):
        pass

#### 1.4 Теперь настало время теста. 
#### Если вы всё сделали правильно, то запустив следующие ячейки у вас должна появиться надпись: Test PASSED

Переходить к дальнейшим заданиям не имеем никакого смысла, пока вы не добьётесь прохождение теста
    

#### Чтение данных

In [1]:
import numpy as np
np.random.seed(123)  # for reproducibility
from keras.utils import np_utils
from keras.datasets import mnist
 
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, 28, 28)
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
 

Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

2022-04-11 16:22:39.397464: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-11 16:22:39.397491: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


(60000, 1, 28, 28) (60000, 10) (10000, 1, 28, 28) (10000, 10)


In [2]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [3]:
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

#### Подготовка моделей

In [8]:
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, Conv2D, MaxPooling2D
from keras.optimizers import gradient_descent_v2
print(keras.__version__)

def get_keras_model():
    input_image = Input(shape=(1, 28, 28))
    flatten = Flatten()(input_image)
    dense1 = Dense(10, activation='softmax')(flatten)
    
    model = Model(inputs=input_image, outputs=dense1)

    sgd = gradient_descent_v2.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(X_train, Y_train, validation_split=0.25, 
                        batch_size=32, epochs=2, verbose=1)
    return model

2.8.0


In [9]:
def get_our_model(keras_model):
    flatten = FlattenLayer()
    dense = DenseLayer(784, 10, W_init=keras_model.get_weights()[0],
                       b_init=keras_model.get_weights()[1])
    softmax = Softmax()
    net = Network([flatten, dense, softmax])
    return net

In [10]:
keras_model = get_keras_model()
our_model = get_our_model(keras_model)

Epoch 1/2


2022-04-01 18:41:16.081352: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-01 18:41:16.081689: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-01 18:41:16.081752: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-04-01 18:41:16.081804: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-04-01 18:41:16.081855: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

1407/1407 [==============================] - 2s 997us/step - loss: 0.4476 - accuracy: 0.8781 - val_loss: 0.3339 - val_accuracy: 0.9061
Epoch 2/2
1407/1407 [==============================] - 1s 661us/step - loss: 0.3250 - accuracy: 0.9085 - val_loss: 0.3062 - val_accuracy: 0.9151


In [11]:
keras_prediction = keras_model.predict(X_test)
our_model_prediction = our_model.predict(X_test)

In [12]:
if np.sum(np.abs(keras_prediction - our_model_prediction)) < 0.01:
    print('Test PASSED')
else:
    print('Something went wrong!')

Test PASSED


#### Тестирование сети со сверточным слоем

In [13]:
X_train_conv = X_train.reshape((X_train.shape[0], X_train.shape[2], X_train.shape[3], X_train.shape[1]))
X_test_conv = X_test.reshape((X_test.shape[0], X_test.shape[2], X_test.shape[3], X_test.shape[1]))[:2500]

In [14]:
def get_keras_conv_model():
    input_image = Input(shape=(28, 28, 1))
    conv2d = Conv2D(filters=3, 
                    kernel_size=3, 
                    strides=1, 
                    padding='same')(input_image)
                    
    flatten = Flatten()(conv2d)
    dense1 = Dense(10, activation='softmax')(flatten)
    model = Model(inputs=input_image, outputs=dense1)

    sgd = gradient_descent_v2.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])

    history = model.fit(X_train_conv, Y_train, validation_split=0.25, 
                        batch_size=32, epochs=2, verbose=1)

    return model

In [15]:
def get_our_conv_model(keras_model):
    conv2d=Conv2DLayer(kernel_size=3, 
                       input_channels=1, 
                       output_channels=3, 
                       padding='same', 
                       stride=1,
                       K_init=keras_model.get_weights()[0],
                       b_init=keras_model.get_weights()[1])
    
    flatten = FlattenLayer()
    dense = DenseLayer(2352, 10, 
                       W_init=keras_model.get_weights()[2],
                       b_init=keras_model.get_weights()[3])
    softmax = Softmax()
    net = Network([conv2d, flatten, dense, softmax])
    return net

In [16]:
keras_model = get_keras_conv_model()
our_model = get_our_conv_model(keras_model)

Epoch 1/2


2022-04-01 18:41:19.226046: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 141120000 exceeds 10% of free system memory.


1407/1407 [==============================] - 2s 2ms/step - loss: 0.3742 - accuracy: 0.8906 - val_loss: 0.3126 - val_accuracy: 0.9113
Epoch 2/2
1407/1407 [==============================] - 2s 2ms/step - loss: 0.3111 - accuracy: 0.9108 - val_loss: 0.3119 - val_accuracy: 0.9114


In [17]:
keras_prediction = keras_model.predict(X_test_conv)
our_model_prediction = our_model.predict(X_test_conv)

In [18]:
if np.sum(np.abs(keras_prediction - our_model_prediction)) < 0.01:
    print('Test PASSED')
else:
    print('Something went wrong!')

Test PASSED


### 2. Вычисление производных по входу для слоёв нейронной сети

В данном задании запрещено использовать численные формулы для вычисления производных.

#### 2.1  Реализуйте метод forward для класса CrossEntropy
Напоминание: $$ crossentropy = L(p, y) =  - \sum\limits_i y_i log p_i, $$
где вектор $(p_1, ..., p_k) $ -  выход классификационного алгоритма, а $(y_1,..., y_k)$ - правильные метки класса в унарной кодировке (one-hot encoding)

In [19]:
class CrossEntropy(object):
    def __init__(self, eps=0.00001):
        self.name = 'CrossEntropy'
        self.eps = eps

    def forward(self, input_data, labels):
        return -np.sum((np.multiply(np.log(input_data), labels)), axis=1)

    def calculate_loss(self, input_data, labels):
        return self.forward(input_data, labels)

    def grad_x(self, input_data, labels):
        return -labels / (input_data + self.eps)

        

#### 2.2  Реализуйте метод grad_x класса CrossEntropy, который возвращает $\frac{\partial L}{\partial p}$

Проверить работоспособность кода поможет следующий тест:

In [20]:
def numerical_diff_net(net, x, labels):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (net.calculate_loss(x + delta, labels) - net.calculate_loss(x-delta, labels)) / (2*eps)
        right_answer.append(diff)
    return np.array(right_answer).T

def test_net(net):
    x = np.array([[1, 2, 3], [2, 3, 4]])
    labels = np.array([[0.3, 0.2, 0.5], [0.3, 0.2, 0.5]])
    num_grad = numerical_diff_net(net, x, labels)
    grad = net.grad_x(x, labels)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)
        
loss = CrossEntropy()
test_net(loss)

Test PASSED


#### 2.3  Реализуйте метод grad_x класса Softmax, который возвращает $\frac{\partial Softmax}{\partial x}$

Проверить работоспособность кода поможет следующий тест:

In [21]:
def numerical_diff_layer(layer, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(x[0])):
        delta = np.zeros(len(x[0]))
        delta[i] = eps
        diff = (layer.forward(x + delta) - layer.forward(x-delta)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_layer(layer):
    x = np.array([[1, 2, 3], [2, -3, 4]])
    num_grad = numerical_diff_layer(layer, x)
    grad = layer.grad_x(x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)
        
layer = Softmax()
test_layer(layer)

Test PASSED


#### 2.4  Реализуйте метод grad_x для классов ReLU и DenseLayer

In [22]:
layer = ReLU()
test_layer(layer)

Test PASSED


In [23]:
layer = DenseLayer(3,4)
test_layer(layer)

Test PASSED


#### 2.5 (4 балла) Для класса Network реализуйте метод grad_x, который должен реализовывать взятие производной от лосса по входу

In [24]:
net = Network([DenseLayer(3, 10), ReLU(), DenseLayer(10, 3), Softmax()], 
              loss=CrossEntropy())
test_net(net)

Test PASSED


### 3. Реализация градиентов по параметрам и метода обратного распространения ошибки с обновлением парметров сети

#### 3.1  Реализуйте функции grad_b и grad_W. При подготовке теста grad_W предполагается, что W является одномерным вектором.

In [25]:
def numerical_grad_b(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(len(b)):
        delta = np.zeros(b.shape)
        delta[i] = eps
        dense1 = DenseLayer(input_size, output_size, W_init=W, b_init=b+delta)
        dense2 = DenseLayer(input_size, output_size, W_init=W, b_init=b-delta)
        diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
        right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_b():
    input_size = 3
    output_size = 4 
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((output_size,))
    x = np.random.random((2, input_size))
    
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_b(x)

    num_grad = numerical_grad_b(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

test_grad_b()

Test PASSED


In [26]:
def numerical_grad_W(input_size, output_size, b, W, x):
    eps = 0.00001
    right_answer = []
    for i in range(W.shape[0]):
        for j in range(W.shape[1]):
            delta = np.zeros(W.shape)
            delta[i, j] = eps
            dense1 = DenseLayer(input_size, output_size, W_init=W+delta, b_init=b)
            dense2 = DenseLayer(input_size, output_size, W_init=W-delta, b_init=b)
            diff = (dense1.forward(x) - dense2.forward(x)) / (2*eps)
            right_answer.append(diff.T)
    return np.array(right_answer).T

def test_grad_W():
    input_size = 3
    output_size = 4 
    W_init = np.random.random((input_size, output_size))
    b_init = np.random.random((4,))
    x = np.random.random((2, input_size))
        
    dense = DenseLayer(input_size, output_size, W_init, b_init)
    grad = dense.grad_W(x)

    num_grad = numerical_grad_W(input_size, output_size, b_init, W_init, x)
    if np.sum(np.abs(num_grad - grad)) < 0.01:
        print('Test PASSED')
    else:
        print('Something went wrong!')
        print('Numerical grad is')
        print(num_grad)
        print('Your gradiend is ')
        print(grad)

test_grad_W()

Test PASSED


#### 3.2 Полностью реализуйте метод обратного распространения ошибки в функции train_step класса Network


Рекомендуем реализовать сначала функцию Network.grad_param(), которая возвращает список длиной в количество слоёв и элементом которого является список градиентов по параметрам.
После чего, имея список градиентов, написать функцию обновления параметров для каждого слоя. 

Совет: рекомендуем написать тест для кода подсчета градиента по параметрам, чтобы быть уверенным в том, что градиент через всю сеть считается правильно
    

#### 3.3 Ознакомьтесь с реализацией функции fit класса Network. Запустите обучение модели. Если всё работает правильно, то точность на валидации должна будет возрастать

In [27]:
net = Network([DenseLayer(784, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::3], Y_train[::3], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.01)

100%|█████████████████████████████████████████| 937/937 [02:04<00:00,  7.55it/s]


1 epoch: val 0.90


100%|█████████████████████████████████████████| 937/937 [02:04<00:00,  7.50it/s]


2 epoch: val 0.91


100%|█████████████████████████████████████████| 937/937 [02:04<00:00,  7.54it/s]


3 epoch: val 0.91


100%|█████████████████████████████████████████| 937/937 [02:04<00:00,  7.53it/s]


4 epoch: val 0.91


100%|█████████████████████████████████████████| 937/937 [02:04<00:00,  7.54it/s]


5 epoch: val 0.92


In [28]:
net = Network([DenseLayer(784, 20), ReLU(), DenseLayer(20, 10), Softmax()], loss=CrossEntropy())
trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::6], Y_train[::6], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.001)    

100%|█████████████████████████████████████████| 468/468 [01:34<00:00,  4.95it/s]


1 epoch: val 0.82


100%|█████████████████████████████████████████| 468/468 [01:35<00:00,  4.92it/s]


2 epoch: val 0.86


100%|█████████████████████████████████████████| 468/468 [01:34<00:00,  4.93it/s]


3 epoch: val 0.87


100%|█████████████████████████████████████████| 468/468 [01:34<00:00,  4.93it/s]


4 epoch: val 0.88


100%|█████████████████████████████████████████| 468/468 [01:34<00:00,  4.94it/s]

5 epoch: val 0.89


#### 3.5 Продемонстрируйте, что ваша реализация позволяет обучать более глубокие нейронные сети 

In [29]:
net = Network([DenseLayer(784, 50), 
               ReLU(), 
               DenseLayer(50, 20), 
               ReLU(), 
               DenseLayer(20, 10), 
               Softmax()], loss=CrossEntropy())

trainX = X_train.reshape(len(X_train), -1)
net.fit(trainX[::6], Y_train[::6], validation_split=0.25, 
            batch_size=16, nb_epoch=5, learning_rate=0.001)    

100%|█████████████████████████████████████████| 468/468 [03:45<00:00,  2.07it/s]


1 epoch: val 0.82


100%|█████████████████████████████████████████| 468/468 [03:44<00:00,  2.09it/s]


2 epoch: val 0.87


100%|█████████████████████████████████████████| 468/468 [03:43<00:00,  2.09it/s]


3 epoch: val 0.89


100%|█████████████████████████████████████████| 468/468 [03:43<00:00,  2.09it/s]


4 epoch: val 0.89


100%|█████████████████████████████████████████| 468/468 [03:43<00:00,  2.09it/s]

5 epoch: val 0.90
